# 7.3 网络中的网络（NiN）
- **目录**
  - 7.3.1 NiN块
  - 7.3.2 NiN模型
  - 7.3.3 训练模型

- LeNet、AlexNet和VGG都有一个共同的设计模式：**通过一系列的卷积层与池化层来提取空间结构特征；然后通过全连接层对特征的表征进行处理**。
- AlexNet和VGG对LeNet的改进主要在于如何扩大和加深这两个模块。
或者，可以想象在这个过程的早期使用全连接层。然而，如果使用了全连接层，可能会完全放弃表征的空间结构。
- **网络中的网络（NiN）** 提供了一个非常简单的解决方案：**在每个像素的通道上分别使用多层感知机。**

## 7.3.1 NiN块

- 卷积层的输入和输出由四维张量组成，张量的**每个轴分别对应样本、通道、高度和宽度**。
- 全连接层的输入和输出通常是分别对应于样本和特征的二维张量。
- NiN的想法是在每个像素位置（针对每个高度和宽度）应用一个全连接层。
- 如果将权重连接到每个空间位置，可以将其视为$1\times 1$卷积层（如 6.4节中所述），或作为在每个像素位置上独立作用的全连接层。
- 从另一个角度看，即将空间维度中的每个像素视为单个样本，将通道维度视为不同特征（feature）。
- 图7.3.1 说明了VGG和NiN及它们的块之间主要架构差异。
- NiN块以一个普通卷积层开始，后面是两个$1 \times 1$的卷积层。这两个$1 \times 1$卷积层充当带有ReLU激活函数的逐像素全连接层。
- 第一层的卷积窗口形状通常由用户设置。随后的卷积窗口形状固定为$1 \times 1$。
<center>
    <img src="../img/nin.svg" alt="对比 VGG 和 NiN 及它们的块之间主要架构差异">
</center>
<center>
    图7.3.1 对比 VGG 和 NiN 及它们的块之间主要架构差异
</center>

In [1]:
%matplotlib inline
import torch
from torch import nn
from d2l import torch as d2l


def nin_block(in_channels, out_channels, kernel_size, strides, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, strides, padding),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU())

## 7.3.2 NiN模型

- 最初的NiN网络是在AlexNet后不久提出的，显然从中得到了一些启示。
- NiN使用窗口形状为$11\times 11$、$5\times 5$和$3\times 3$的卷积层，输出通道数量与AlexNet中的相同。
- 每个NiN块后有一个最大池化层，池化窗口形状为$3\times 3$，步幅为2。
- NiN和AlexNet之间的一个显著区别是NiN完全取消了全连接层。
- 相反，NiN使用一个NiN块，其输出通道数等于标签类别的数量。
- 最后放一个**全局平均池化层（global average pooling layer）**，生成一个对数几率：（logits）。
- NiN设计的一个优点是，它显著减少了模型所需参数的数量。然而，在实践中，这种设计有时会增加训练模型的时间。


In [2]:
net = nn.Sequential(
    nin_block(1, 96, kernel_size=11, strides=4, padding=0),
    nn.MaxPool2d(3, stride=2),
    nin_block(96, 256, kernel_size=5, strides=1, padding=2),
    nn.MaxPool2d(3, stride=2),
    nin_block(256, 384, kernel_size=3, strides=1, padding=1),
    nn.MaxPool2d(3, stride=2),
    nn.Dropout(0.5),
    # 标签类别数是10
    nin_block(384, 10, kernel_size=3, strides=1, padding=1),
    nn.AdaptiveAvgPool2d((1, 1)),
    # 将四维的输出转成二维的输出，其形状为(批量大小,10)
    nn.Flatten())

- 创建一个数据样本来查看每个块的输出形状。


In [3]:
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 96, 54, 54])
MaxPool2d output shape:	 torch.Size([1, 96, 26, 26])
Sequential output shape:	 torch.Size([1, 256, 26, 26])
MaxPool2d output shape:	 torch.Size([1, 256, 12, 12])
Sequential output shape:	 torch.Size([1, 384, 12, 12])
MaxPool2d output shape:	 torch.Size([1, 384, 5, 5])
Dropout output shape:	 torch.Size([1, 384, 5, 5])
Sequential output shape:	 torch.Size([1, 10, 5, 5])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 10, 1, 1])
Flatten output shape:	 torch.Size([1, 10])


## 7.3.3 训练模型

- 仍使用Fashion-MNIST来训练模型。训练NiN与训练AlexNet、VGG时相似。


In [7]:
import time
import numpy as np
lr, num_epochs, batch_size = 0.1, 10, 128
#lr, num_epochs, batch_size = 0.1, 2, 32
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, resize=224)
start = time.time()
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())
end = time.time()
print(f'3090ti上模型训练耗时：{int(np.floor((end-start)/60))}分钟{(int(end-start)%60) }秒')

<img src="../img/7_3_1.png" height=350 width=350>

------------

- **说明：**
- **（1）1x1 卷积核在NiN中的作用**
  
  - **背景与动机**
    - **传统卷积的局限性**：传统的卷积层通过固定大小的卷积核（如 3x3、5x5）提取局部空间特征，但在捕获复杂的非线性特征组合方面存在局限。
    - **增强特征表达能力**：NiN 的设计目标是通过更复杂的特征映射来增强网络的表达能力，而不仅仅依赖于简单的线性卷积。
  - **1x1 卷积的基本功能**
    - **通道间的线性组合**：1x1 卷积核在每个像素位置上对输入特征图的所有通道进行线性组合。这相当于在每个像素位置上执行一个全连接操作。
    - **特征重组**：通过这种线性组合，1x1 卷积可以重新组合和整合不同通道的信息，形成新的特征表示。
  - **1x1 卷积在 NiN 中的具体作用**
    - **逐像素的非线性变换**：在 NiN 中，1x1 卷积核不仅用于线性组合，还通过后续的非线性激活函数（如 ReLU）实现逐像素的非线性变换。这种变换使得网络能够学习更复杂的特征表示。
    - **增强网络深度和非线性**：通过在每个卷积层后添加 1x1 卷积层，NiN 增加了网络的深度和非线性能力，使得网络能够更好地拟合复杂的函数。
  -  **1x1 卷积的优势**
    - **参数效率**：相比于增加更多的卷积层或更大的卷积核，1x1 卷积增加的参数相对较少，但能显著提升模型的表达能力。
    - **降维和升维**：1x1 卷积可以用于降维（减少通道数）以降低计算复杂度，也可以用于升维（增加通道数）以增强特征表达。
    - **局部感受野的全局化**：通过在局部感受野上应用 1x1 卷积，NiN 能够在不增加感受野大小的情况下增强特征的全局性。
  - **NiN 网络架构中的应用**
    - **MLPConv 层**：NiN 引入了 MLPConv（多层感知器卷积）层，其中每个卷积层后跟随一个 1x1 卷积层，形成一个小型的多层感知器。这种设计使得每个卷积层不仅仅是线性卷积，而是一个小型的神经网络。
    - **全局平均池化**：在 NiN 的最后，使用全局平均池化代替全连接层，进一步减少参数数量，并利用 1x1 卷积的特性来整合全局信息。

- **（2）NiN架构深入分析**
  - **网络中的网络（Network in Network，简称NiN）** 是一种卷积神经网络（CNN）架构，于2013年提出。
  - NiN提出了一个重要思想：在卷积层后面添加多层感知器（MLP），从而更好地获取局部特征。
  - 在传统的CNN中，卷积层负责提取空间特征，池化层负责降低空间尺寸以减少计算量。
  - NiN通过使用1x1的卷积核来代替全连接层，将多层感知器应用于局部区域，进一步提取抽象特征。
  - 以下是NiN模型的各层架构和数据形状：
    - 1. **输入层**：
      - 输入数据形状：(h, w, c)，其中h, w表示图像高度和宽度，c表示通道数（例如，3表示RGB颜色通道）。

    - 2. **卷积层1（Conv1）**：
      - 参数：96个11x11卷积核，步长为4。
      - 输出数据形状：((h-11)/4+1, (w-11)/4+1, 96)。

    - 3. **MLP1**：
      - 参数：256个1x1卷积核。
      - 输出数据形状：((h-11)/4+1, (w-11)/4+1, 256)。

    - 4. **MLP2**：
      - 参数：384个1x1卷积核。
      - 输出数据形状：((h-11)/4+1, (w-11)/4+1, 384)。

    - 5. **最大池化层1（MaxPool1）**：
      - 参数：3x3窗口大小，步长为2。
      - 输出数据形状：((((h-11)/4+1)-3)/2+1, (((w-11)/4+1)-3)/2+1, 384)。

    - 6. **卷积层2（Conv2）**：
      - 参数：384个5x5卷积核，填充为2。
      - 输出数据形状：((((h-11)/4+1)-3)/2+1, (((w-11)/4+1)-3)/2+1, 384)。

    - 7. **MLP3**：
      - 参数：384个1x1卷积核。
      - 输出数据形状：((((h-11)/4+1)-3)/2+1, (((w-11)/4+1)-3)/2+1, 384)。

    - 8. **MLP4**：
      - 参数：256个1x1卷积核。
      - 输出数据形状：((((h-11)/4+1)-3)/2+1, (((w-11)/4+1)-3)/2+1, 256)。

    - 9. **最大池化层2（MaxPool2）**：
      - 参数：3x3窗口大小，步长为2。
      - 输出数据形状：((((((h-11)/4+1)-3)/2+1)-3)/2+1, (((((w-11)/4+1)-3)/2+1)-3)/2+1, 256)。

    - 10. **卷积层3（Conv3）**：
      - 参数：256个3x3卷积核，填充为1。
      - 输出数据形状：((((((h-11)/4+1)-3)/2+1)-3)/2+1, (((((w-11)/4+1)-3)/2+1)-3)/2+1, 256)。

    - 11. **MLP5**：
      - 参数：num_classes个1x1卷积核，其中`num_classes`表示目标类别数。
      - 输出数据形状：((((((h-11)/4+1)-3)/2+1)-3)/2+1, (((((w-11)/4+1)-3)/2+1)-3)/2+1, num_classes)。

    - 12.**全局平均池化层（Global Average Pooling）**：
      - 参数：无。
      - 输出数据形状：(1, 1, num_classes)。全局平均池化层的目的是将每个特征图降维到一个数值，即对每个通道进行全局空间平均。这样可以有效减少参数数量并减轻过拟合现象。

    - 13. **输出层（Output）**：
      - 参数：无。
      - 输出数据形状：(num_classes)。通过去掉空间维度，输出一个长度为num_classes的向量，表示每个类别的预测分数。在NiN模型中，激活函数通常是ReLU（Rectified Linear Unit）。在全局平均池化层之后，可以使用Softmax激活函数进行多类别分类。

    - **总结**：NiN模型包含3个卷积层（Conv1、Conv2和Conv3），5个MLP层（MLP1至MLP5），2个最大池化层（MaxPool1和MaxPool2），以及一个全局平均池化层。NiN通过在卷积层后添加1x1卷积核的多层感知器来提取更抽象的特征，从而实现网络中的网络结构。此外，NiN使用全局平均池化层代替传统的全连接层，有效地减少了参数数量并降低过拟合风险。

------

## 小结

* NiN使用由一个卷积层和多个$1\times 1$卷积层组成的块。该块可以在卷积神经网络中使用，以允许更多的每像素非线性。
* NiN去除了容易造成过拟合的全连接层，将它们替换为全局平均池化层（即在所有位置上进行求和）。该池化层通道数量为所需的输出数量（例如，Fashion-MNIST的输出为10）。
* 移除全连接层可减少过拟合，同时显著减少NiN的参数。
* NiN的设计影响了许多后续卷积神经网络的设计。

